In [1]:
import cv2
import numpy as np
import pandas as pd
import datetime 

# Functions need

In [2]:
def get_center(x, y, w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy


In [3]:
vechile_detector = cv2.createBackgroundSubtractorMOG2()
'''-------------------------'''
min_Width = 60
min_Hight = 60
detected = []
detected_wh = []


line_Pos_Right = 605
line_Pos_Left = 90

cars_Left = 0
cars_Right = 0
trucks_Left = 0
trucks_Right = 0
offset = 6

'''-------------------------'''
begin = datetime.datetime.now()
cars_Left_ = 0
cars_Right_ = 0
trucks_Left_ = 0
trucks_Right_ = 0
tL = []
tR = []
cL = []
cR = []
mydict = {0: 0}

'''-------------------------'''

video = 'video2.AVI'
cap = cv2.VideoCapture(video)
while True:

    ret, frame = cap.read()
    if not ret:
        break
    height, width, depth = frame.shape

    frame = cv2.resize(frame, (750, 750))
    orignal = frame
    frame = frame[0:750, 50:750]
    frame = cv2.resize(frame, (750, 750))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(frame, (7, 7), 5)
    mask = vechile_detector.apply(blur)
    _, mask = cv2.threshold(mask, 253, 255, cv2.THRESH_BINARY)

    countershape, h = cv2.findContours(
        mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #cv2.line(frame, (line_Pos_Left, 250), (line_Pos_Left, 750), (0, 0, 255), 3)
    #cv2.line(frame, (line_Pos_Right, 250),(line_Pos_Right, 750), (0, 0, 255), 3)

    for i, c in enumerate(countershape):
        x, y, w, h = cv2.boundingRect(c)
        valid = (w >= min_Width) and (h >= min_Hight)
        if not valid:
            continue
        cv2.rectangle(orignal, (x, y), (x+w, y+h), (0, 255, 0), 2)

        center = get_center(x, y, w, h)
        #cv2.circle(orignal, center, 4, (0, 0, 255), -1)

        detected.append(center)
        detected_wh.append((w, h))
        for i, (x, y) in enumerate(detected):
            w, h = detected_wh[i]
            if x < line_Pos_Left + offset and x > line_Pos_Left-offset:
                if w >= 160 and h >= 160:
                    trucks_Left += 1
                    trucks_Left_ += 1

                else:
                    cars_Left += 1
                    cars_Left_ += 1
                #cv2.line(frame, (line_Pos_Left, 250),(line_Pos_Left, 750), (0, 255,0), 3)

            if x < line_Pos_Right + offset and x > line_Pos_Right-offset:
                if w >= 160 and h >= 160:
                    trucks_Right += 1
                    trucks_Right_ += 1
                else:
                    cars_Right += 1
                    cars_Right_ += 1
                #cv2.line(frame, (line_Pos_Right, 250),(line_Pos_Right, 750), (0, 255,0), 3)
            detected.remove((x, y))
            detected_wh.remove((w, h))

    s = "Right trucks: "+str(trucks_Right)
    cv2.putText(orignal, s, (450, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    s = "Right cars: "+str(cars_Right)
    cv2.putText(orignal, s, (450, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    s = "Left trucks: "+str(trucks_Left)
    cv2.putText(orignal, s, (50, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    s = "Left cars: "+str(cars_Left)
    cv2.putText(orignal, s, (50, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    cv2.imshow("Video", orignal)

    cur_time = datetime.datetime.now()
    tot_min = cur_time - begin
    tot_min = tot_min.seconds

    if str(int(tot_min)) not in mydict.keys() and tot_min % 60 == 0 and tot_min > 0:
        print(tot_min)
        mydict[str(tot_min)] = '1'
        tL.append(trucks_Left_)
        tR.append(trucks_Right_)
        cL.append(cars_Left_)
        cR.append(cars_Right_)
        cars_Left_, trucks_Left_, cars_Right_, trucks_Right_ = 0, 0, 0, 0

    if cv2.waitKey(30) == ord('1'):
        break

cap.release()
cv2.destroyAllWindows()


60
120


In [4]:
mydict = {}
n = len(tR)
arrfrom = [i for i in range(0, n, 1)]
arrto = [x + 1 for x in arrfrom]
arrtot = [cR[i]+cL[i]+tL[i]+tR[i] for i in range(n)]

mydict['From'] = arrfrom
mydict['To'] = arrto
mydict['Cars right'] = cR
mydict['Cars left'] = cL
mydict['Trucks right'] = tR
mydict['Trucks left'] = tL
mydict['Total vehicles'] = arrtot

df = pd.DataFrame(mydict)
df


,From,To,Cars right,Cars left,Trucks right,Trucks left,Total vehicles
0,0,1,3,4,0,0,7
1,1,2,5,6,0,1,12


In [5]:
_ = df.to_excel('excel_file.xlsx', index=False)